### Bildvorverarbeitung

In [4]:
import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass
import random
import shutil
import sys

@dataclass
class QRROIConfig:
    patch_size: int = 256
    roi_overlap: float = 0.75
    global_overlap: float = 0.75
    min_area: int = 256
    top_k: int = 50
    enable_global_search: bool = True
    global_scale_divisor: int = 4
    min_adaptive_size: int = 128
    debug_view: bool = True 

# --- Hilfsfunktionen (Unverändert) ---

def get_square_patch(img, cx, cy, size, target_size=256):
    half = size // 2
    x0, y0 = cx - half, cy - half
    x1, y1 = x0 + size, y0 + size
    h, w = img.shape[:2]
    pad_top = max(0, -y0); pad_bottom = max(0, y1 - h)
    pad_left = max(0, -x0); pad_right = max(0, x1 - w)
    
    if any([pad_top, pad_bottom, pad_left, pad_right]):
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0,0,0])
        x0 += pad_left; x1 += pad_left; y0 += pad_top; y1 += pad_top
        patch = img_padded[y0:y1, x0:x1]
    else:
        patch = img[y0:y1, x0:x1]
        
    if patch.shape[0] != target_size or patch.shape[1] != target_size:
        patch = cv2.resize(patch, (target_size, target_size), interpolation=cv2.INTER_LANCZOS4)
    return patch

def generate_roi_patches(img, cand, cfg: QRROIConfig):
    patches, coords = [], []
    S = cfg.patch_size
    if cand["w"] <= S and cand["h"] <= S:
        crop_size = max(cand["w"], cand["h"])
        patches.append(get_square_patch(img, cand["cx"], cand["cy"], crop_size, S))
        coords.append((cand["cx"] - crop_size//2, cand["cy"] - crop_size//2, crop_size))
    else:
        stride = max(1, int(S * (1 - cfg.roi_overlap)))
        for y_s in range(cand["y"], cand["y"] + cand["h"] - S + stride, stride):
            for x_s in range(cand["x"], cand["x"] + cand["w"] - S + stride, stride):
                ax, ay = min(x_s, cand["x"] + cand["w"] - S), min(y_s, cand["y"] + cand["h"] - S)
                patches.append(get_square_patch(img, ax + S//2, ay + S//2, S, S))
                coords.append((ax, ay, S))
    return patches, coords

def generate_global_patches(img, cfg: QRROIConfig):
    h, w = img.shape[:2]
    patches, coords = [], []
    win_size = max(cfg.min_adaptive_size, min(w, h) // cfg.global_scale_divisor)
    stride = max(1, int(win_size * (1 - cfg.global_overlap)))
    for y in range(0, h - win_size + stride, stride):
        for x in range(0, w - win_size + stride, stride):
            cx, cy = min(x, w - win_size), min(y, h - win_size)
            patches.append(get_square_patch(img, cx + win_size//2, cy + win_size//2, win_size, cfg.patch_size))
            coords.append((cx, cy, win_size))
    return patches, coords

def detect_candidates(img, cfg: QRROIConfig):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for cnt in contours:
        if cv2.contourArea(cnt) < cfg.min_area: continue
        x, y, w, h = cv2.boundingRect(cnt)
        candidates.append({"cx": x + w // 2, "cy": y + h // 2, "x": x, "y": y, "w": w, "h": h})
    return sorted(candidates, key=lambda x: (x["w"]*x["h"]), reverse=True)[:cfg.top_k]

# --- Hauptfunktion ---

def main():
    # --- 1. Pfad-Konfiguration ---
    input_dir = Path("~/DatenUbuntu/Studium/1. Semester/KI-Projekt/modeltest/pictures").expanduser()         # Quelle (Alle Bilder)
    test_picture_dir = Path("test_picture") # Ziel für Original-Kopien
    output_root = Path("test_patches")    # Ziel für Patches & Metadata
    
    cfg = QRROIConfig()
    
    # Ordner erstellen
    test_picture_dir.mkdir(parents=True, exist_ok=True)
    out_patches_root = output_root / "patches"
    out_patches_root.mkdir(parents=True, exist_ok=True)

    # --- 2. Bilder finden & Auswahl ---
    if not input_dir.exists():
        print(f"❌ Ordner '{input_dir}' existiert nicht.")
        return

    img_files = list(input_dir.glob("*.[jJ][pP][gG]")) + list(input_dir.glob("*.[pP][nN][gG]"))
    
    if not img_files:
        print(f"❌ Keine Bilder in '{input_dir}' gefunden.")
        return

    try:
        user_input = input(f"Wie viele Bilder aus '{len(img_files)}' zufällig wählen? (Zahl oder 'all'): ")
        if user_input.lower() == 'all':
            num_to_process = len(img_files)
        else:
            num_to_process = int(user_input)
            num_to_process = max(1, min(num_to_process, len(img_files)))
    except ValueError:
        print("⚠️ Ungültige Eingabe. Verarbeite 1 Bild.")
        num_to_process = 1

    selected_files = random.sample(img_files, num_to_process)
    patch_metadata = []

    print(f"🚀 Starte Verarbeitung von {num_to_process} Bildern...")

    # --- 3. Verarbeitungsschleife ---
    for img_file in selected_files:
        # A) Originalbild kopieren
        shutil.copy2(img_file, test_picture_dir / img_file.name)
        
        # B) Bild laden
        img = cv2.imread(str(img_file))
        if img is None: continue
        
        # Unterordner für Patches erstellen
        img_patch_dir = out_patches_root / img_file.stem
        img_patch_dir.mkdir(parents=True, exist_ok=True)
        
        h_orig, w_orig = img.shape[:2]
        vis_img = img.copy()
        
        # C) ROI Patches (Kandidaten)
        candidates = detect_candidates(img, cfg)
        for i, cand in enumerate(candidates):
            roi_ps, roi_coords = generate_roi_patches(img, cand, cfg)
            cv2.rectangle(vis_img, (cand["x"], cand["y"]), (cand["x"]+cand["w"], cand["y"]+cand["h"]), (0, 255, 0), 4)
            
            for p_idx, (p, (px, py, ps)) in enumerate(zip(roi_ps, roi_coords)):
                name = f"ROI_{i}_p{p_idx}.jpg"
                cv2.imwrite(str(img_patch_dir / name), p)
                
                # Metadata schreiben
                patch_metadata.append(f"{img_file.name};{img_file.stem}/{name};{px};{py};{ps};{w_orig};{h_orig}")
                
                cv2.rectangle(vis_img, (px, py), (px+ps, py+ps), (0, 255, 255), 2)

        # D) Globale Patches (Sliding Window über alles)
        if cfg.enable_global_search:
            global_ps, global_coords = generate_global_patches(img, cfg)
            for g_idx, (gp, (gx, gy, gs)) in enumerate(zip(global_ps, global_coords)):
                name = f"GLOBAL_p{g_idx}.jpg"
                cv2.imwrite(str(img_patch_dir / name), gp)
                
                patch_metadata.append(f"{img_file.name};{img_file.stem}/{name};{gx};{gy};{gs};{w_orig};{h_orig}")
                
                cv2.rectangle(vis_img, (gx, gy), (gx+gs, gy+gs), (0, 0, 255), 1)

        print(f"-> {img_file.name}: Verarbeitet und Patches gespeichert.")

        # E) Vorschau (Optional)
        if cfg.debug_view:
            h, w = vis_img.shape[:2]
            scale = 800 / max(h, w)
            res_small = cv2.resize(vis_img, (int(w * scale), int(h * scale)))
            cv2.imshow("Preprocessing Vorschau (Auto-Run)", res_small)
            # 100ms warten, dann weiter. Mit 'q' abbrechen.
            if cv2.waitKey(100) & 0xFF == ord('q'): 
                break

    # Metadata im Hauptordner speichern
    with open(output_root / "metadata.txt", "w") as f:
        f.write("\n".join(patch_metadata))
    
    cv2.destroyAllWindows()
    print(f"✅ Fertig! Originale in '{test_picture_dir}', Ergebnisse in '{output_root}'.")

if __name__ == "__main__":
    main()

🚀 Starte Verarbeitung von 100 Bildern...
-> Passat_N2_frame1830.jpg: Verarbeitet und Patches gespeichert.
-> Passat_P1_frame2330.jpg: Verarbeitet und Patches gespeichert.
-> FocusP3_frame_3420.jpg: Verarbeitet und Patches gespeichert.
-> CKlasse_Bilderreihe 4_P_IMG_9644_rot-11.jpg: Verarbeitet und Patches gespeichert.
-> Focus_P2_frame_3120.jpg: Verarbeitet und Patches gespeichert.
-> CKlasse_Bilderreihe 2_P_IMG_9462_noise.jpg: Verarbeitet und Patches gespeichert.
-> Focus_P2_frame_3660.jpg: Verarbeitet und Patches gespeichert.
-> Passat_P2_frame1540.jpg: Verarbeitet und Patches gespeichert.
-> IMG_6739.jpg: Verarbeitet und Patches gespeichert.
-> CKlasse_Bilderreihe 2_P_IMG_9478_rot-2.jpg: Verarbeitet und Patches gespeichert.
-> Golf_IV_P_2.jpg: Verarbeitet und Patches gespeichert.
-> Passat_N3_frame1320.jpg: Verarbeitet und Patches gespeichert.
-> Focus_P1_frame_1440.jpg: Verarbeitet und Patches gespeichert.
-> Seat_P_37.jpg: Verarbeitet und Patches gespeichert.
-> FocusP4_frame_1260

### Test Model

In [5]:
import os

# 1. System-Konfiguration (Unterdrückt Warnungen und behebt Berechtigungsprobleme)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['MPLCONFIGDIR'] = os.path.join(os.getcwd(), "tmp_matplotlib_cache")
if not os.path.exists(os.environ['MPLCONFIGDIR']):
    os.makedirs(os.environ['MPLCONFIGDIR'], exist_ok=True)

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from ipywidgets import interact, IntSlider

# 2. Pfade und Parameter (Basierend auf deiner Struktur)
model_path = 'final_model.keras'
folder_path = 'test_patches/patches'
img_size = (256, 256)

# 3. Modell laden
try:
    model = load_model(model_path)
    print(f"✅ Modell '{model_path}' erfolgreich geladen.")
except Exception as e:
    print(f"❌ Fehler beim Laden des Modells: {e}")
    model = None

# 4. Bilder-Liste erstellen
if os.path.exists(folder_path):
    files = sorted([str(p.relative_to(folder_path)) for p in Path(folder_path).rglob('*') 
                    if p.suffix.lower() in ('.png', '.jpg', '.jpeg', '.tif')])
    print(f"🔍 {len(files)} Bilder in Unterordnern von '{folder_path}' gefunden.")
else:
    print(f"❌ Ordner '{folder_path}' wurde nicht gefunden!")
    files = []

# 5. Anzeige-Funktion für den Slider
def browse_patches(index):
    if not files:
        print("Keine Bilder vorhanden.")
        return

    filename = files[index]
    img_path = os.path.join(folder_path, filename)
    
    # Bild laden (Grayscale + 256x256)
    img = load_img(img_path, target_size=img_size, color_mode='grayscale')
    img_array = img_to_array(img)
    
    img_tensor = np.expand_dims(img_array, axis=0)

    # Vorhersage
    prediction = model.predict(img_tensor, verbose=0)
    
    # Bestimmung der Wahrscheinlichkeit für Klasse 1
    if prediction.shape[-1] == 1:
        prob_1 = float(prediction[0][0])
    else:
        prob_1 = float(prediction[0][1])
    
    label = 1 if prob_1 > 0.5 else 0
    color = 'green' if label == 1 else 'red'

    # Visualisierung
    plt.figure(figsize=(6, 6))
    plt.imshow(img_array.squeeze(), cmap='gray')
    
    title_str = (f"Bild {index+1}/{len(files)}: {filename}\n"
                 f"KLASSE: {label} | Wahrsch. Klasse 1: {prob_1:.4f}")
    
    plt.title(title_str, color=color, fontsize=12, fontweight='bold')
    plt.axis('off')
    plt.show()

# 6. Interaktives Element starten
if files and model:
    print("Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:")
    interact(browse_patches, index=IntSlider(
        min=0, 
        max=len(files)-1, 
        step=1, 
        value=0, 
        description='Patch-Index:',
        layout={'width': '500px'}
    ))

/home/marlon/DatenUbuntu/Github/KI-Projekt/KI-Project-WS2526/.venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
I0000 00:00:1767907454.426736  223777 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1207 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


✅ Modell 'final_model.keras' erfolgreich geladen.
🔍 68346 Bilder in Unterordnern von 'test_patches/patches' gefunden.
Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:


interactive(children=(IntSlider(value=0, description='Patch-Index:', layout=Layout(width='500px'), max=68345),…

In [1]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
import gc
from tensorflow.keras.models import load_model

# --- 1. GPU & System Setup (Robust) ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Wichtig: Erlaubt TensorFlow, Speicher dynamisch zu nehmen
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 
os.environ['MPLCONFIGDIR'] = str(Path.home() / ".matplotlib_cache")
Path(os.environ['MPLCONFIGDIR']).mkdir(exist_ok=True)

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                try:
                    tf.config.experimental.set_memory_growth(gpu, True)
                except:
                    pass
            print(f"✅ GPU Beschleunigung aktiv.")
    except RuntimeError:
        print("ℹ️ GPU Setup übersprungen.")
setup_gpu()

# --- 2. Konfiguration ---
class QRFinalConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    original_img_dir = Path('test_picture')
    output_dir = Path('final_results')
    
    threshold = 0.85      
    start_batch_size = 32  # Wir versuchen es erst schnell mit 32

# --- 3. Die kombinierte Pipeline ---
def run_combined_pipeline():
    cfg = QRFinalConfig()
    cfg.output_dir.mkdir(exist_ok=True)
    
    if not cfg.metadata_file.exists():
        print("❌ Metadaten-Datei nicht gefunden!")
        return

    # compile=False ist wichtig, spart Speicher
    model = load_model(cfg.model_path, compile=False)
    
    with open(cfg.metadata_file, "r") as f:
        lines = [l.strip().split(";") for l in f.readlines()]

    images_dict = {}
    for img_name, rel_path, px, py, ps, w_orig, h_orig in lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({"path": rel_path, "x": int(px), "y": int(py), "s": int(ps)})

    print(f"--- Analyse startet (Versuche Batch-Size: {cfg.start_batch_size}) ---")

    for img_name, info in images_dict.items():
        orig_img = cv2.imread(str(cfg.original_img_dir / img_name))
        if orig_img is None: continue
        h_orig, w_orig = info["h"], info["w"]
        
        patch_list = info["patches"]
        if not patch_list: continue

        all_patch_imgs = []
        for p in patch_list:
            p_img = cv2.imread(str(cfg.patch_folder / p["path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256), interpolation=cv2.INTER_LINEAR)
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs: continue

        # Daten vorbereiten
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)
        
        # --- INTELLIGENTE VORHERSAGE ---
        # Hier ist der Fix: Wir probieren 32. Wenn Crash -> 16 -> 8 -> ...
        preds = None
        current_bs = cfg.start_batch_size
        
        while current_bs >= 1:
            try:
                preds = model.predict(input_batch, batch_size=current_bs, verbose=0)
                break # Erfolg! Raus aus der Schleife
            except tf.errors.ResourceExhaustedError:
                # print(f"⚠️ Speicher voll bei BS {current_bs}, reduziere auf {current_bs//2}...")
                current_bs //= 2
                gc.collect() # Speicher aufräumen
                if current_bs < 1:
                    print(f"❌ Bild {img_name} ist zu groß für den VRAM.")
                    preds = None
                    break
        
        if preds is None: continue

        # --- HEATMAP REKONSTRUKTION ---
        heatmap_max = np.zeros((h_orig, w_orig), dtype=np.float32)
        max_prob = 0.0
        
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            max_prob = max(max_prob, prob)
            
            p = patch_list[i]
            x, y, s = p["x"], p["y"], p["s"]
            heatmap_max[y:y+s, x:x+s] = np.maximum(heatmap_max[y:y+s, x:x+s], prob)

        # Speicher freigeben für nächstes Bild
        del input_batch, all_patch_imgs, preds
        gc.collect()

        # ==========================================================
        # VISUALISIERUNG & LOGIK (Wie in deinem Originalcode)
        # ==========================================================
        _, thresh = cv2.threshold(heatmap_max, cfg.threshold, 1.0, cv2.THRESH_BINARY)
        thresh_8bit = (thresh * 255).astype(np.uint8)
        
        contours, _ = cv2.findContours(thresh_8bit, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        heatmap_8bit = (heatmap_max * 255).astype(np.uint8)
        heatmap_vis = cv2.GaussianBlur(heatmap_8bit, (15, 15), 0)
        heatmap_color = cv2.applyColorMap(heatmap_vis, cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(orig_img, 0.6, heatmap_color, 0.4, 0)
        
        found_count = 0
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 15 and h > 15:
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 255, 0), 4)
                found_count += 1

        status = "QR GEFUNDEN" if max_prob >= cfg.threshold else "KEIN TREFFER"
        cv2.rectangle(overlay, (0, 0), (overlay.shape[1], 60), (0, 0, 0), -1)
        
        # Info anzeigen (inklusive genutzter Batch Size, falls reduziert)
        bs_info = "" if current_bs == cfg.start_batch_size else f"(BS:{current_bs})"
        cv2.putText(overlay, f"{status} | Prob: {max_prob:.1%} {bs_info}", 
                    (20, 42), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        cv2.imwrite(str(cfg.output_dir / f"final_{img_name}"), overlay)
        print(f"✅ {img_name}: {status} ({max_prob:.2%}) {bs_info}")

    tf.keras.backend.clear_session()

if __name__ == "__main__":
    run_combined_pipeline()

2026-01-08 22:38:30.227448: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-08 22:38:30.269053: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-08 22:38:31.025790: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
mkdir -p failed for path /mnt/DatenUbuntu/marlon_home/.cache/matplotlib: [Errno 13] Permiss

✅ GPU Beschleunigung aktiv.


I0000 00:00:1767908312.427907  275707 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4081 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


--- Analyse startet (Versuche Batch-Size: 32) ---


2026-01-08 22:38:33.728987: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701


✅ Passat_N2_frame1830.jpg: KEIN TREFFER (42.50%) 
✅ Passat_P1_frame2330.jpg: QR GEFUNDEN (99.78%) 
✅ FocusP3_frame_3420.jpg: QR GEFUNDEN (100.00%) 
✅ CKlasse_Bilderreihe 4_P_IMG_9644_rot-11.jpg: QR GEFUNDEN (100.00%) 
✅ Focus_P2_frame_3120.jpg: QR GEFUNDEN (100.00%) 
✅ CKlasse_Bilderreihe 2_P_IMG_9462_noise.jpg: QR GEFUNDEN (100.00%) 
✅ Focus_P2_frame_3660.jpg: QR GEFUNDEN (100.00%) 
✅ Passat_P2_frame1540.jpg: QR GEFUNDEN (100.00%) 
✅ IMG_6739.jpg: QR GEFUNDEN (99.73%) 
✅ CKlasse_Bilderreihe 2_P_IMG_9478_rot-2.jpg: QR GEFUNDEN (100.00%) 
✅ Golf_IV_P_2.jpg: QR GEFUNDEN (99.99%) 
✅ Passat_N3_frame1320.jpg: KEIN TREFFER (50.34%) 
✅ Focus_P1_frame_1440.jpg: QR GEFUNDEN (100.00%) 
✅ Seat_P_37.jpg: QR GEFUNDEN (100.00%) 
✅ FocusP4_frame_1260.jpg: QR GEFUNDEN (100.00%) 
✅ Focus_P2_frame_3670.jpg: QR GEFUNDEN (100.00%) 
✅ Focus_P1_frame_650.jpg: QR GEFUNDEN (100.00%) 
✅ Passat_P3_frame3650.jpg: QR GEFUNDEN (100.00%) 
✅ 20251213_125053_037.jpg: QR GEFUNDEN (100.00%) 
✅ CKlasse_Bilderreihe 2_P_I

## Heatmap

In [2]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- 1. Robustes GPU-Setup ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                if not tf.config.experimental.get_memory_growth(gpu):
                    tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU (RTX 4050) für Heatmap-Erstellung bereit.")
    except RuntimeError:
        print("ℹ️ GPU war bereits initialisiert. Fahre fort...")

setup_gpu()

# --- 2. Konfiguration ---
class HeatmapConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    output_folder = Path('heatmaps_output')
    
    batch_size = 32  # Passt gut in den 6GB VRAM der RTX 4050

# --- 3. Hauptfunktion ---
def generate_gpu_heatmaps():
    cfg = HeatmapConfig()
    cfg.output_folder.mkdir(exist_ok=True)

    if not cfg.metadata_file.exists():
        print(f"❌ {cfg.metadata_file} nicht gefunden!")
        return

    # Modell laden
    print("⏳ Lade Modell...")
    model = load_model(cfg.model_path)

    # Metadaten einlesen
    with open(cfg.metadata_file, "r") as f:
        data_lines = [line.strip().split(";") for line in f.readlines()]

    # Gruppierung nach Originalbild
    images_dict = {}
    for img_name, rel_patch_path, px, py, ps, w_orig, h_orig in data_lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({
            "rel_path": rel_patch_path, "x": int(px), "y": int(py), "s": int(ps)
        })

    print(f"--- Starte Heatmap-Generierung für {len(images_dict)} Bilder ---")

    for img_name, info in images_dict.items():
        h, w = info["h"], info["w"]
        # Initialisierung mit Nullen
        heatmap_max = np.zeros((h, w), dtype=np.float32)

        print(f"🚀 Verarbeite: {img_name}")

        # --- BATCH LOADING ---
        patch_list = info["patches"]
        all_patch_imgs = []
        
        for p in patch_list:
            # Schnelles Laden via OpenCV (Graustufen, 0-255 Scale)
            p_img = cv2.imread(str(cfg.patch_folder / p["rel_path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256))
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs:
            continue

        # In Tensor-Format umwandeln (Batch, Height, Width, Channels)
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)

        # --- GPU INFERENCE ---
        # Alle Patches des Bildes in einem Rutsch vorhersagen
        preds = model.predict(input_batch, batch_size=cfg.batch_size, verbose=0)

        # --- REKONSTRUKTION ---
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            
            p = patch_list[i]
            px, py, ps = p["x"], p["y"], p["s"]
            
            # Maximum-Logik (Pfad A): Der höchste Wert pro Pixel gewinnt
            # Das verhindert das "Verwässern" durch schlechte Patches
            heatmap_max[py:py+ps, px:px+ps] = np.maximum(heatmap_max[py:py+ps, px:px+ps], prob)

        # --- SPEICHERN ---
        # Umwandlung in 8-Bit Grayscale
        heatmap_8bit = (heatmap_max * 255).astype(np.uint8)
        
        # Optional: Falls du eine farbige Heatmap willst, aktiviere die nächste Zeile:
        # heatmap_8bit = cv2.applyColorMap(heatmap_8bit, cv2.COLORMAP_JET)

        output_path = cfg.output_folder / f"{Path(img_name).stem}_heatmap.png"
        cv2.imwrite(str(output_path), heatmap_8bit)

    print(f"✅ Alle Heatmaps wurden in '{cfg.output_folder}' erstellt.")

# --- Startbefehl ---
if __name__ == "__main__":
    generate_gpu_heatmaps()

✅ GPU (RTX 4050) für Heatmap-Erstellung bereit.
⏳ Lade Modell...
--- Starte Heatmap-Generierung für 100 Bilder ---
🚀 Verarbeite: Passat_N2_frame1830.jpg
🚀 Verarbeite: Passat_P1_frame2330.jpg
🚀 Verarbeite: FocusP3_frame_3420.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 4_P_IMG_9644_rot-11.jpg
🚀 Verarbeite: Focus_P2_frame_3120.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 2_P_IMG_9462_noise.jpg
🚀 Verarbeite: Focus_P2_frame_3660.jpg
🚀 Verarbeite: Passat_P2_frame1540.jpg
🚀 Verarbeite: IMG_6739.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 2_P_IMG_9478_rot-2.jpg
🚀 Verarbeite: Golf_IV_P_2.jpg
🚀 Verarbeite: Passat_N3_frame1320.jpg
🚀 Verarbeite: Focus_P1_frame_1440.jpg
🚀 Verarbeite: Seat_P_37.jpg
🚀 Verarbeite: FocusP4_frame_1260.jpg
🚀 Verarbeite: Focus_P2_frame_3670.jpg
🚀 Verarbeite: Focus_P1_frame_650.jpg
🚀 Verarbeite: Passat_P3_frame3650.jpg
🚀 Verarbeite: 20251213_125053_037.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 2_P_IMG_9394_rot-1.jpg
🚀 Verarbeite: Seat_P_19.jpg
🚀 Verarbeite: Passat_P2_frame2680.jpg
🚀 Verarbeite: 

### Vergleich der erzeugten Ergebnisse

In [1]:
import cv2
import numpy as np
from pathlib import Path

# --- 1. Konfiguration der Pfade ---
path_original = Path('test_picture')
path_heatmap = Path('heatmaps_output')
path_decision = Path('final_results')

# --- 2. FIXE FENSTERGRÖSSE ---
WINDOW_WIDTH = 1500
WINDOW_HEIGHT = 500

def show_interactive_evaluation():
    # Bilder suchen
    img_files = sorted(list(path_original.glob("*.[jJ][pP][gG]")) + list(path_original.glob("*.[pP][nN][gG]")))
    
    if not img_files:
        print("❌ Keine Originalbilder gefunden.")
        return

    single_w = WINDOW_WIDTH // 3
    single_h = WINDOW_HEIGHT

    # --- COUNTER INITIALISIEREN ---
    stats = {
        "richtig": 0,
        "falsch": 0,
        "gesamt": 0
    }

    print(f"--- Interaktive Evaluation gestartet ---")
    print("Steuerung: [J] = Richtig erkannt | [N] = Falsch erkannt | [Q] = Beenden")

    for idx, img_file in enumerate(img_files):
        stem = img_file.stem
        
        # 1. Bilder laden
        img_orig = cv2.imread(str(img_file))
        
        # Heatmap suchen (verschiedene Endungen)
        heatmap_path = next((p for p in [path_heatmap / f"{stem}_heatmap.png", path_heatmap / f"{stem}_heatmap.jpg"] if p.exists()), None)
        img_heat = cv2.imread(str(heatmap_path)) if heatmap_path else None
        
        # Ergebnis suchen
        final_path = next((p for p in [path_decision / f"final_{img_file.name}", path_decision / f"result_{img_file.name}"] if p.exists()), None)
        img_final = cv2.imread(str(final_path)) if final_path else None

        # Falls ein Teil fehlt, überspringen
        if img_orig is None or img_heat is None or img_final is None:
            continue

        # 2. Resizing
        res_orig = cv2.resize(img_orig, (single_w, single_h), interpolation=cv2.INTER_AREA)
        res_heat = cv2.resize(img_heat, (single_w, single_h), interpolation=cv2.INTER_AREA)
        res_final = cv2.resize(img_final, (single_w, single_h), interpolation=cv2.INTER_AREA)

        # 3. Farbraum-Check
        if len(res_heat.shape) == 2:
            res_heat = cv2.cvtColor(res_heat, cv2.COLOR_GRAY2BGR)

        # 4. Horizontal stapeln
        triple_view = np.hstack((res_orig, res_heat, res_final))

        # 5. Beschriftungen (Header)
        header_text = [("ORIGINAL", 10), ("HEATMAP", single_w + 10), ("DETEKTION", (single_w * 2) + 10)]
        for text, pos_x in header_text:
            cv2.putText(triple_view, text, (pos_x + 2, 32), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
            cv2.putText(triple_view, text, (pos_x, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # 6. Status-Leiste (Footer)
        # Balken unten für bessere Lesbarkeit
        cv2.rectangle(triple_view, (0, single_h - 40), (WINDOW_WIDTH, single_h), (50, 50, 50), -1)
        
        status_str = f"Bild {idx+1}/{len(img_files)} | Richtig: {stats['richtig']} | Falsch: {stats['falsch']} | DRUECKE: [J]a / [N]ein"
        cv2.putText(triple_view, status_str, (20, single_h - 12), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 7. Anzeige & Interaktions-Loop
        cv2.imshow("QR-Analyse Evaluation", triple_view)
        
        valid_key = False
        while not valid_key:
            key = cv2.waitKey(0) & 0xFF
            
            if key == ord('q'):
                print("\n🛑 Abbruch durch Benutzer.")
                print_summary(stats)
                cv2.destroyAllWindows()
                return

            elif key == ord('j'): # Ja / Richtig
                stats["richtig"] += 1
                stats["gesamt"] += 1
                print(f"✅ Bild {idx+1}: Richtig markiert.")
                valid_key = True

            elif key == ord('n'): # Nein / Falsch
                stats["falsch"] += 1
                stats["gesamt"] += 1
                print(f"❌ Bild {idx+1}: Falsch markiert.")
                valid_key = True
            
            else:
                print("⚠️  Bitte 'j' für Richtig, 'n' für Falsch oder 'q' zum Beenden drücken.")

    print_summary(stats)
    cv2.destroyAllWindows()

def print_summary(stats):
    total = stats["gesamt"]
    if total == 0:
        print("Keine Bilder bewertet.")
        return
        
    acc = (stats["richtig"] / total) * 100
    print("\n" + "="*40)
    print(f"📊 ABSCHLUSSBERICHT")
    print("="*40)
    print(f"Gesamt bewertet: {total}")
    print(f"✅ Richtig:       {stats['richtig']}")
    print(f"❌ Falsch:        {stats['falsch']}")
    print("-" * 40)
    print(f"📈 Genauigkeit:   {acc:.2f}%")
    print("="*40)

if __name__ == "__main__":
    show_interactive_evaluation()

--- Interaktive Evaluation gestartet ---
Steuerung: [J] = Richtig erkannt | [N] = Falsch erkannt | [Q] = Beenden
✅ Bild 1: Richtig markiert.
✅ Bild 2: Richtig markiert.
✅ Bild 3: Richtig markiert.
✅ Bild 4: Richtig markiert.
✅ Bild 5: Richtig markiert.
✅ Bild 6: Richtig markiert.
❌ Bild 7: Falsch markiert.
✅ Bild 8: Richtig markiert.
✅ Bild 9: Richtig markiert.
✅ Bild 10: Richtig markiert.
✅ Bild 11: Richtig markiert.
✅ Bild 12: Richtig markiert.
✅ Bild 13: Richtig markiert.
✅ Bild 14: Richtig markiert.
✅ Bild 15: Richtig markiert.
✅ Bild 16: Richtig markiert.
✅ Bild 17: Richtig markiert.
✅ Bild 18: Richtig markiert.
✅ Bild 19: Richtig markiert.
✅ Bild 20: Richtig markiert.
✅ Bild 21: Richtig markiert.
✅ Bild 22: Richtig markiert.
✅ Bild 23: Richtig markiert.
✅ Bild 24: Richtig markiert.
✅ Bild 25: Richtig markiert.
✅ Bild 26: Richtig markiert.
✅ Bild 27: Richtig markiert.
✅ Bild 28: Richtig markiert.
✅ Bild 29: Richtig markiert.
✅ Bild 30: Richtig markiert.
✅ Bild 31: Richtig markiert